# Groupby and Arrest Data

In [1]:
#import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt

# Exercise 1

In [2]:
#define results dictionary
results = {}

In [3]:
#load data
df = pd.read_csv("https://media.githubusercontent.com/media/nickeubank/MIDS_Data/master/OnlineArrestData1980-2021.csv")
df.head(10)

,YEAR,GENDER,RACE,AGE_GROUP,COUNTY,VIOLENT,PROPERTY,F_DRUGOFF,F_SEXOFF,F_ALLOTHER,F_TOTAL,M_TOTAL,S_TOTAL
0,1980,Male,Black,Under 18,Alameda County,505,1351,188,26,79,2149,2286,295
1,1980,Male,Black,18 to 19,Alameda County,205,465,183,8,48,909,1333,0
2,1980,Male,Black,20 to 29,Alameda County,949,1593,606,27,178,3353,7974,0
3,1980,Male,Black,30 to 39,Alameda County,450,755,241,18,110,1574,4876,0
4,1980,Male,Black,40 to 69,Alameda County,172,218,117,11,66,584,3836,0
5,1980,Male,Black,70 and over,Alameda County,4,2,2,0,0,8,79,0
6,1980,Male,Hispanic,Under 18,Alameda County,151,365,40,9,77,642,1303,187
7,1980,Male,Hispanic,18 to 19,Alameda County,83,89,48,0,58,278,963,0
8,1980,Male,Hispanic,20 to 29,Alameda County,226,243,158,13,86,726,3748,0
9,1980,Male,Hispanic,30 to 39,Alameda County,59,80,41,8,33,221,1669,0


# Exercise 2

From the table, each row represents how many people of a certain gender, race and age group in a certain county and year commited a particular type of crime. 

# Exericse 3

In [4]:
df1 = df.duplicated(["YEAR", "GENDER", "RACE", "AGE_GROUP", "COUNTY"])
assert not df1.any()

# Exercise 4

In [5]:
county_year_race = df.groupby(["YEAR", "RACE", "COUNTY"], as_index=False).sum()
county_year_race

/var/folders/n2/5x35hhp53jg1kc9jxswsqhww0000gn/T/ipykernel_51296/3573147655.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  county_year_race = df.groupby(["YEAR", "RACE", "COUNTY"], as_index=False).sum()


,YEAR,RACE,COUNTY,VIOLENT,PROPERTY,F_DRUGOFF,F_SEXOFF,F_ALLOTHER,F_TOTAL,M_TOTAL,S_TOTAL
0,1980,Black,Alameda County,2594,5138,1489,94,541,9856,24081,555
1,1980,Black,Butte County,32,33,5,0,7,77,139,0
2,1980,Black,Calaveras County,0,0,0,0,0,0,7,0
3,1980,Black,Colusa County,0,5,0,0,0,5,13,0
4,1980,Black,Contra Costa County,720,1422,368,24,257,2791,5201,40
...,...,...,...,...,...,...,...,...,...,...,...
9697,2021,White,Tulare County,524,407,195,10,413,1549,3368,31
9698,2021,White,Tuolumne County,182,63,35,13,289,582,1393,2
9699,2021,White,Ventura County,574,372,308,13,672,1939,7509,18
9700,2021,White,Yolo County,171,163,51,8,173,566,1051,0


In [9]:
results['ex4_num_rows'] = county_year_race.shape[0]
print(f"There are {results['ex4_num_rows']} rows present in the grouped dataframe.")

There are 9702 rows present in the grouped dataframe.


# Exercise 5

In [11]:
results['ex5_collapsed_vars'] = ["AGE_GROUP", "GENDER"]
results['ex5_collapsed_vars']

['AGE_GROUP', 'GENDER']

# Exercise 6

In [15]:
#manually checking values
alameda_1980 = county_year_race[(county_year_race["COUNTY"] == "Alameda County") & (county_year_race["YEAR"] == 1980)]
print(f"total violent crimes in Alameda County in 1980: {sum(alameda_1980['VIOLENT'])}")
print(f"total drug crimes in Alameda County in 1980: {sum(alameda_1980['F_DRUGOFF'])}")

total violent crimes in Alameda County in 1980: 4504
total drug crimes in Alameda County in 1980: 3569


In [14]:
violent_year = county_year_race.groupby(['COUNTY', 'YEAR'])['VIOLENT'].transform('sum')
drug_year = county_year_race.groupby(['COUNTY', 'YEAR'])['F_DRUGOFF'].transform('sum')
county_year_race['total_violent'] = violent_year
county_year_race['total_drug'] = drug_year
county_year_race

,YEAR,RACE,COUNTY,VIOLENT,PROPERTY,F_DRUGOFF,F_SEXOFF,F_ALLOTHER,F_TOTAL,M_TOTAL,S_TOTAL,total_violent,total_drug
0,1980,Black,Alameda County,2594,5138,1489,94,541,9856,24081,555,4504,3569
1,1980,Black,Butte County,32,33,5,0,7,77,139,0,306,149
2,1980,Black,Calaveras County,0,0,0,0,0,0,7,0,52,83
3,1980,Black,Colusa County,0,5,0,0,0,5,13,0,28,9
4,1980,Black,Contra Costa County,720,1422,368,24,257,2791,5201,40,1596,1089
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9697,2021,White,Tulare County,524,407,195,10,413,1549,3368,31,2360,635
9698,2021,White,Tuolumne County,182,63,35,13,289,582,1393,2,229,44
9699,2021,White,Ventura County,574,372,308,13,672,1939,7509,18,1845,752
9700,2021,White,Yolo County,171,163,51,8,173,566,1051,0,480,92


# Exercise 7

In [16]:
county_year_race['violent_share'] = county_year_race['VIOLENT'] / county_year_race['total_violent']
county_year_race['drug_share'] = county_year_race['F_DRUGOFF'] / county_year_race['total_drug']
county_year_race

,YEAR,RACE,COUNTY,VIOLENT,PROPERTY,F_DRUGOFF,F_SEXOFF,F_ALLOTHER,F_TOTAL,M_TOTAL,S_TOTAL,total_violent,total_drug,violent_share,drug_share
0,1980,Black,Alameda County,2594,5138,1489,94,541,9856,24081,555,4504,3569,0.575933,0.417204
1,1980,Black,Butte County,32,33,5,0,7,77,139,0,306,149,0.104575,0.033557
2,1980,Black,Calaveras County,0,0,0,0,0,0,7,0,52,83,0.000000,0.000000
3,1980,Black,Colusa County,0,5,0,0,0,5,13,0,28,9,0.000000,0.000000
4,1980,Black,Contra Costa County,720,1422,368,24,257,2791,5201,40,1596,1089,0.451128,0.337925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9697,2021,White,Tulare County,524,407,195,10,413,1549,3368,31,2360,635,0.222034,0.307087
9698,2021,White,Tuolumne County,182,63,35,13,289,582,1393,2,229,44,0.794760,0.795455
9699,2021,White,Ventura County,574,372,308,13,672,1939,7509,18,1845,752,0.311111,0.409574
9700,2021,White,Yolo County,171,163,51,8,173,566,1051,0,480,92,0.356250,0.554348


In [25]:
alameda_1980_w_share = county_year_race[(county_year_race["COUNTY"] == "Alameda County") & (county_year_race["YEAR"] == 1980)]
print(sum(alameda_1980_w_share['violent_share']))
sum(alameda_1980_w_share['drug_share']) #both should sum to 1

1.0


1.0

In [41]:
results['ex7_alameda_1980_share_violent_arrestees_black'] = alameda_1980_w_share.loc[(alameda_1980_w_share['RACE'] == 'Black'), 'violent_share'].item()
results['ex7_alameda_1980_share_violent_arrestees_black']

0.5759325044404974

# Exercise 8

In [85]:
check_shares = county_year_race.groupby(['COUNTY', 'YEAR'], as_index=False)[['violent_share', 'drug_share']].sum()
assert len(check_shares['violent_share'].to_numpy()) == np.sum(check_shares['violent_share'].to_numpy())
county_year_race.iloc[check_shares['drug_share'][check_shares['drug_share'] == 0].index.values]
county_year_race[(county_year_race['COUNTY'] == 'Solano County') & (county_year_race['YEAR'] == 'Solano 1980')]

,YEAR,RACE,COUNTY,VIOLENT,PROPERTY,F_DRUGOFF,F_SEXOFF,F_ALLOTHER,F_TOTAL,M_TOTAL,S_TOTAL,total_violent,total_drug,violent_share,drug_share
44,1980,Black,Solano County,214,483,63,7,84,851,1752,20,624,397,0.342949,0.158690
45,1980,Black,Sonoma County,57,88,15,2,11,173,404,12,555,336,0.102703,0.044643
47,1980,Black,Sutter County,1,5,0,0,2,8,43,0,128,54,0.007812,0.000000
48,1980,Black,Tehama County,0,2,0,0,0,2,7,0,87,29,0.000000,0.000000
78,1980,Hispanic,Modoc County,3,5,0,1,0,9,16,0,18,5,0.166667,0.000000
82,1980,Hispanic,Nevada County,1,4,3,0,1,9,31,0,70,93,0.014286,0.032258


In [97]:
county_year_race[(county_year_race['COUNTY'] == 'Alpine County') & (county_year_race['YEAR'].isin([1982, 1983, 1985, 1986, 2016, 2020]))]

,YEAR,RACE,COUNTY,VIOLENT,PROPERTY,F_DRUGOFF,F_SEXOFF,F_ALLOTHER,F_TOTAL,M_TOTAL,S_TOTAL,total_violent,total_drug,violent_share,drug_share
569,1982,Other,Alpine County,1,3,0,0,1,5,13,0,1,0,1.000000,NaN
627,1982,White,Alpine County,0,13,0,1,1,15,28,0,1,0,0.000000,NaN
742,1983,Hispanic,Alpine County,0,0,0,0,0,0,1,0,2,0,0.000000,NaN
800,1983,Other,Alpine County,0,0,0,0,0,0,2,0,2,0,0.000000,NaN
858,1983,White,Alpine County,2,5,0,0,5,12,21,0,2,0,1.000000,NaN
1258,1985,Other,Alpine County,2,2,0,0,1,5,5,0,2,0,1.000000,NaN
1316,1985,White,Alpine County,0,9,0,0,3,12,28,1,2,0,0.000000,NaN
1486,1986,Other,Alpine County,5,6,0,0,0,11,12,0,10,0,0.500000,NaN
1544,1986,White,Alpine County,5,7,0,0,2,14,32,0,10,0,0.500000,NaN
8311,2016,Black,Alpine County,0,0,0,0,0,0,2,0,5,0,0.000000,NaN


In [100]:
np.unique(county_year_race[county_year_race['drug_share'].isna()]['YEAR'].to_numpy())

array([1982, 1983, 1985, 1986, 2016, 2020])

For years [1982, 1983, 1985, 1986, 2016, 2020] in Alpine County there were 0 felony drug arrests so when taking the share it was entered as NaN. Instead, we will impute these as 0. However, our original assert that we used for violent_crimes could not be applied here since there was no crime to be reported. We should expect there to be 6 less shares of drug_shares to account for the six years in Alpine County where there were no reported felony drug arrests. 

In [116]:
county_year_race.loc[county_year_race['drug_share'].isna(), 'drug_share'] = 0
county_year_race.loc[county_year_race['drug_share'].isna()]
assert (len(check_shares['drug_share'].to_numpy()) - 6) == np.sum(check_shares['drug_share'].to_numpy())

In [48]:
num_black_counties = len(county_year_race.loc[county_year_race['RACE'] == 'Black', 'COUNTY'].unique())
num_white_counties = len(county_year_race.loc[county_year_race['RACE'] == 'White', 'COUNTY'].unique())
num_hispanic_counties = len(county_year_race.loc[county_year_race['RACE'] == 'Hispanic', 'COUNTY'].unique())
num_other_counties = len(county_year_race.loc[county_year_race['RACE'] == 'Other', 'COUNTY'].unique())
#58 counties in California according to Google
assert all(i == 58 for i in [num_other_counties, num_hispanic_counties, num_black_counties, num_white_counties])

# Exercise 9

In [137]:
counties_yearrace = county_year_race.groupby(['YEAR', 'RACE'], as_index=False)['COUNTY'].count()
counties_yearrace[counties_yearrace['COUNTY'] != 58]

,YEAR,RACE,COUNTY
0,1980,Black,55
1,1980,Hispanic,57
4,1981,Black,55
5,1981,Hispanic,57
8,1982,Black,55
9,1982,Hispanic,57
12,1983,Black,57
16,1984,Black,55
17,1984,Hispanic,57
20,1985,Black,57


In [143]:
a = set(county_year_race[(county_year_race['RACE'] == 'Black') & (county_year_race['YEAR'] == 1980)]['COUNTY'])
b = set(county_year_race[(county_year_race['RACE'] == 'White') & (county_year_race['YEAR'] == 1980)]['COUNTY'])
b.difference(a)

{'Alpine County', 'Amador County', 'Sierra County'}